In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/KS_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2766427408,KS,"4910 Corporate Centre Dr. Suite 210, LAWRENCE, KS",38.953377,-95.309460,2.004500e+14
1,3354047105,KS,"509 E ELM ST, SALINA, KS",38.845120,-97.604220,2.016900e+14
2,5028487210,KS,"8415 E 21st St North Ste 100, WICHITA, KS",37.723057,-97.440765,2.017301e+14
3,9101897101,KS,"301 W 13th Street, HAYS, KS",38.874940,-99.332695,2.005107e+14
4,9638147110,KS,"13795 S Murlen Ste 301, OLATHE, KS",38.877364,-94.778463,2.009105e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2766427408,KS,"4910 Corporate Centre Dr. Suite 210, LAWRENCE, KS",38.953377,-95.309460,2.004500e+14
1,3354047105,KS,"509 E ELM ST, SALINA, KS",38.845120,-97.604220,2.016900e+14
2,5028487210,KS,"8415 E 21st St North Ste 100, WICHITA, KS",37.723057,-97.440765,2.017301e+14
3,9101897101,KS,"301 W 13th Street, HAYS, KS",38.874940,-99.332695,2.005107e+14
4,9638147110,KS,"13795 S Murlen Ste 301, OLATHE, KS",38.877364,-94.778463,2.009105e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
22,6801687203,KS,"1902 S US Highway 69, Parsons, KS",32.699351,-95.507551,NaN
74,5701067007,KS,"1201 E MAIN ST, TOWANDA, KS",35.987549,-78.887411,NaN
79,9919287004,KS,"1331 N. US HIGHWAY 81 BYP, MCPHERSON, KS",48.410695,-97.407804,NaN
354,2105228406,KS,"1331 N US Highway 81 Byp, McPherson, KS",48.410695,-97.407804,NaN
531,2828358610,KS,"106 N 6th St, Garden City, KS",40.722950,-73.636430,NaN
713,9291238806,KS,"427 W main street Gardner KS 66030, Gardner, KS",36.209095,-94.802270,NaN
1245,8004737201,KS,"212 Main Street, PO Box 127, MINNEOLA, KS",39.190140,-94.683716,NaN
1343,4279937301,KS,"1002 South 4th St, KIOWA, KS",40.591825,-75.459935,NaN
1349,9823617002,KS,"1010 E HWY 54, LIBERAL, KS",47.947866,-116.790779,NaN
1354,4447707008,KS,"1201 S. Main St. P.O. Box 528, MCPHERSON, KS",39.195324,-94.683390,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('20')] #KS
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/31 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

31


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10855/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
22,6801687203,KS,"1902 S US Highway 69, Parsons, KS",32.699351,-95.507551,[]
74,5701067007,KS,"1201 E MAIN ST, TOWANDA, KS",35.987549,-78.887411,[]
79,9919287004,KS,"1331 N. US HIGHWAY 81 BYP, MCPHERSON, KS",48.410695,-97.407804,[]
354,2105228406,KS,"1331 N US Highway 81 Byp, McPherson, KS",48.410695,-97.407804,[]
531,2828358610,KS,"106 N 6th St, Garden City, KS",40.722950,-73.636430,[]
713,9291238806,KS,"427 W main street Gardner KS 66030, Gardner, KS",36.209095,-94.802270,[]
1245,8004737201,KS,"212 Main Street, PO Box 127, MINNEOLA, KS",39.190140,-94.683716,[]
1343,4279937301,KS,"1002 South 4th St, KIOWA, KS",40.591825,-75.459935,[]
1349,9823617002,KS,"1010 E HWY 54, LIBERAL, KS",47.947866,-116.790779,[]
1354,4447707008,KS,"1201 S. Main St. P.O. Box 528, MCPHERSON, KS",39.195324,-94.683390,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10855/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
22,6801687203,KS,"1902 S US Highway 69, Parsons, KS",32.699351,-95.507551,NaN
74,5701067007,KS,"1201 E MAIN ST, TOWANDA, KS",35.987549,-78.887411,NaN
79,9919287004,KS,"1331 N. US HIGHWAY 81 BYP, MCPHERSON, KS",48.410695,-97.407804,NaN
354,2105228406,KS,"1331 N US Highway 81 Byp, McPherson, KS",48.410695,-97.407804,NaN
531,2828358610,KS,"106 N 6th St, Garden City, KS",40.722950,-73.636430,NaN
713,9291238806,KS,"427 W main street Gardner KS 66030, Gardner, KS",36.209095,-94.802270,NaN
1245,8004737201,KS,"212 Main Street, PO Box 127, MINNEOLA, KS",39.190140,-94.683716,NaN
1343,4279937301,KS,"1002 South 4th St, KIOWA, KS",40.591825,-75.459935,NaN
1349,9823617002,KS,"1010 E HWY 54, LIBERAL, KS",47.947866,-116.790779,NaN
1354,4447707008,KS,"1201 S. Main St. P.O. Box 528, MCPHERSON, KS",39.195324,-94.683390,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
22,6801687203,KS,"1902 S US Highway 69, Parsons, KS",32.699351,-95.507551,NaN
74,5701067007,KS,"1201 E MAIN ST, TOWANDA, KS",35.987549,-78.887411,NaN
79,9919287004,KS,"1331 N. US HIGHWAY 81 BYP, MCPHERSON, KS",48.410695,-97.407804,NaN
354,2105228406,KS,"1331 N US Highway 81 Byp, McPherson, KS",48.410695,-97.407804,NaN
531,2828358610,KS,"106 N 6th St, Garden City, KS",40.722950,-73.636430,NaN
713,9291238806,KS,"427 W main street Gardner KS 66030, Gardner, KS",36.209095,-94.802270,NaN
1245,8004737201,KS,"212 Main Street, PO Box 127, MINNEOLA, KS",39.190140,-94.683716,NaN
1343,4279937301,KS,"1002 South 4th St, KIOWA, KS",40.591825,-75.459935,NaN
1349,9823617002,KS,"1010 E HWY 54, LIBERAL, KS",47.947866,-116.790779,NaN
1354,4447707008,KS,"1201 S. Main St. P.O. Box 528, MCPHERSON, KS",39.195324,-94.683390,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
22,6801687203,NaN
74,5701067007,NaN
79,9919287004,NaN
354,2105228406,NaN
531,2828358610,NaN
713,9291238806,NaN
1245,8004737201,NaN
1343,4279937301,NaN
1349,9823617002,NaN
1354,4447707008,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/KS_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,365496,2766427408,KS,"4910 Corporate Centre Dr. Suite 210, LAWRENCE, KS",38.953377,-95.309460,2.004500e+14
1,365497,3354047105,KS,"509 E ELM ST, SALINA, KS",38.845120,-97.604220,2.016900e+14
2,365498,5028487210,KS,"8415 E 21st St North Ste 100, WICHITA, KS",37.723057,-97.440765,2.017301e+14
3,365499,9101897101,KS,"301 W 13th Street, HAYS, KS",38.874940,-99.332695,2.005107e+14
4,365500,9638147110,KS,"13795 S Murlen Ste 301, OLATHE, KS",38.877364,-94.778463,2.009105e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,365496,2766427408,KS,"4910 Corporate Centre Dr. Suite 210, LAWRENCE, KS",38.953377,-95.309460,2.004500e+14,NaN
1,365497,3354047105,KS,"509 E ELM ST, SALINA, KS",38.845120,-97.604220,2.016900e+14,NaN
2,365498,5028487210,KS,"8415 E 21st St North Ste 100, WICHITA, KS",37.723057,-97.440765,2.017301e+14,NaN
3,365499,9101897101,KS,"301 W 13th Street, HAYS, KS",38.874940,-99.332695,2.005107e+14,NaN
4,365500,9638147110,KS,"13795 S Murlen Ste 301, OLATHE, KS",38.877364,-94.778463,2.009105e+14,NaN
...,...,...,...,...,...,...,...,...
8196,373692,9051777907,KS,"2200 W 6th Avenue, EL DORADO, KS",37.824745,-96.878530,2.001502e+14,NaN
8197,373693,9799577002,KS,"9939 NW US HIGHWAY 24, SILVER LAKE, KS",39.094315,-95.851010,2.017700e+14,NaN
8198,373694,9833288500,KS,"8112 W Central Ave, Wichita, KS",37.694230,-97.436295,2.017301e+14,NaN
8199,373695,9895717007,KS,"12548 US HIGHWAY 54, ANDOVER, KS",37.679720,-97.074350,2.001502e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,365496,2766427408,KS,"4910 Corporate Centre Dr. Suite 210, LAWRENCE, KS",38.953377,-95.309460,2.004500e+14,NaN,2.004500e+14
1,365497,3354047105,KS,"509 E ELM ST, SALINA, KS",38.845120,-97.604220,2.016900e+14,NaN,2.016900e+14
2,365498,5028487210,KS,"8415 E 21st St North Ste 100, WICHITA, KS",37.723057,-97.440765,2.017301e+14,NaN,2.017301e+14
3,365499,9101897101,KS,"301 W 13th Street, HAYS, KS",38.874940,-99.332695,2.005107e+14,NaN,2.005107e+14
4,365500,9638147110,KS,"13795 S Murlen Ste 301, OLATHE, KS",38.877364,-94.778463,2.009105e+14,NaN,2.009105e+14


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2766427408,KS,"4910 Corporate Centre Dr. Suite 210, LAWRENCE, KS",38.953377,-95.309460,2.004500e+14
1,3354047105,KS,"509 E ELM ST, SALINA, KS",38.845120,-97.604220,2.016900e+14
2,5028487210,KS,"8415 E 21st St North Ste 100, WICHITA, KS",37.723057,-97.440765,2.017301e+14
3,9101897101,KS,"301 W 13th Street, HAYS, KS",38.874940,-99.332695,2.005107e+14
4,9638147110,KS,"13795 S Murlen Ste 301, OLATHE, KS",38.877364,-94.778463,2.009105e+14
...,...,...,...,...,...,...
8196,9051777907,KS,"2200 W 6th Avenue, EL DORADO, KS",37.824745,-96.878530,2.001502e+14
8197,9799577002,KS,"9939 NW US HIGHWAY 24, SILVER LAKE, KS",39.094315,-95.851010,2.017700e+14
8198,9833288500,KS,"8112 W Central Ave, Wichita, KS",37.694230,-97.436295,2.017301e+14
8199,9895717007,KS,"12548 US HIGHWAY 54, ANDOVER, KS",37.679720,-97.074350,2.001502e+14


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
22,6801687203,KS,"1902 S US Highway 69, Parsons, KS",32.699351,-95.507551,NaN
74,5701067007,KS,"1201 E MAIN ST, TOWANDA, KS",35.987549,-78.887411,NaN
79,9919287004,KS,"1331 N. US HIGHWAY 81 BYP, MCPHERSON, KS",48.410695,-97.407804,NaN
354,2105228406,KS,"1331 N US Highway 81 Byp, McPherson, KS",48.410695,-97.407804,NaN
531,2828358610,KS,"106 N 6th St, Garden City, KS",40.722950,-73.636430,NaN
713,9291238806,KS,"427 W main street Gardner KS 66030, Gardner, KS",36.209095,-94.802270,NaN
1245,8004737201,KS,"212 Main Street, PO Box 127, MINNEOLA, KS",39.190140,-94.683716,NaN
1343,4279937301,KS,"1002 South 4th St, KIOWA, KS",40.591825,-75.459935,NaN
1349,9823617002,KS,"1010 E HWY 54, LIBERAL, KS",47.947866,-116.790779,NaN
1354,4447707008,KS,"1201 S. Main St. P.O. Box 528, MCPHERSON, KS",39.195324,-94.683390,NaN


Convert to File

In [19]:
fips_KS = fips_merge_drop

In [20]:
fips_KS.to_csv("../../../data/state_data/geo/geo_fips/23/KS_fips_scraped.csv") 